In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File"  # file without extention
sheets = "85 Sentence"  # 2 gram target, 2 gram hybrid
file_ext = "85 Sentence"
first_column = 2  # start column location number
end_column = 3

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# word all usage in sent
word_use_big_than = 1

In [4]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [5]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
# path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
# Deployment2/Result/4-Adjust Word Group In Youtube Sentence Word Usage Analysis2/{lang_folder.capitalize()}"
# 
# Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [8]:
df_file = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name=f"{sheets}")
#df_file.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=True)
df_file.reset_index(drop=True, inplace=True)
df_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
1,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"
2,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
3,çok iyi değil mi tam böyle gerçek gibi gerçek ...,3,6,çok iyi değil mi tam böyle gerçek gibi gerçek ...,OmT_BtZh8qU,https://www.youtube.com/watch?v=OmT_BtZh8qU&t=3s,"[9, 28, 14, 5, 112, 56, 183, 27, 183, 27, 60]"
4,nasıl bir şekilde olduğunu ve neden o şekilde ...,14,17,şimdi apexin nasıl bir şekilde olduğunu ve ned...,owj1QZoYqFs,https://www.youtube.com/watch?v=owj1QZoYqFs&t=14s,"[39, 0, 181, 40, 3, 34, 6, 181, 40]"
...,...,...,...,...,...,...,...
80,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
81,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"
82,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...,"[48, 81, 48, 81]"
83,bir gece çok kötü bir,822,824,hayat mücadelesi verdiğimde bir gece çok kötü ...,eS84Pe23HMA,https://www.youtube.com/watch?v=eS84Pe23HMA&t=...,NaN


In [9]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > word_use_big_than]
check_list = df_check["word"].to_list()
remove_list = []
df_file_var = df_file
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file_var = df_file_var[df_file_var["search_string"] != f"{var_sent}"]
        df_file_var_count = word_count_result(df_file_var, df_file_var.columns[0:1])
        df_check = df_file_var_count[df_file_var_count["word_count"] > word_use_big_than]
        check_list = df_check["word"].to_list()
        remove_list.append(var_sent)
    else:
        pass

df_file_var.reset_index(drop=True, inplace=True)
df_file_var

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
1,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"
2,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
3,bile yok bu da benim için aslında önemli bir ş...,131,134,bulabilirsiniz şu an görünce bile yok bu da be...,aqQ7IUho9pA,https://www.youtube.com/watch?v=aqQ7IUho9pA&t=...,"[59, 22, 1, 15, 31, 4, 174, 156, 0, 16, 68]"
4,için artık değil misin gerek yok bunun için si...,65,70,göndermek için artık değil misin gerek yok bun...,gX77n13Qj0o,https://www.youtube.com/watch?v=gX77n13Qj0o&t=65s,"[4, 61, 14, 49, 117, 22, 97, 4, 121, 0]"
...,...,...,...,...,...,...,...
57,diye söyle bak hala,104,106,o taşlarını gecesi zor için yoğurduktan sonra ...,P5bYtFrqw68,https://www.youtube.com/watch?v=P5bYtFrqw68&t=...,"[123, 161, 64, 167]"
58,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s,"[58, 193, 51, 23]"
59,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
60,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"


In [10]:
remove_list

['çok iyi değil mi tam böyle gerçek gibi gerçek gibi yani',
 'nasıl bir şekilde olduğunu ve neden o şekilde olduğunu',
 'burada böyle hayır öyle bir şey olabilir mi asla asla',
 'bence çok güzel bence çok güzel senin zaten',
 'biraz daha devam et devam et tamam oldu',
 'kız seni gel buraya gel buraya kız gel',
 'bakalım son olarak neler var neler',
 'hemen ve her gün her dakika oluyor',
 'aslında seninle birlikte orada',
 'bakalım nasıl olacak ve buna',
 'çünkü hiçbir zaman bana göre',
 'ile hemen hemen aynı hiçbir',
 'onları biliyor musun çünkü',
 'size söyle merhaba merhaba',
 'geldi ona kadar ona kadar',
 'olsun ki sadece biz bize',
 'bize lazım olacak çünkü',
 'biri büyük biri küçük',
 'tabii onlar tabii ona',
 'olan oldu artık geri',
 'tam başka en uzun',
 'şu adam şu adam',
 'tamam peki pekala pekala tamam']

In [11]:
df_result_word_count = word_count_result(df_file_var, df_file.columns[0:1])
df_result_word_count

,word,word_count
0,bir,13
1,çok,11
2,değil,7
3,için,7
4,bakalım,6
...,...,...
187,herkes,1
188,hemen,1
189,hayır,1
190,haydi,1


In [12]:
df_result_word_count["word_count"].sum()

395

In [17]:
df_file_var.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx", index=False)

##### Not In Sent And Result Word Count

In [13]:
set_result_word_count = set(df_result_word_count["word"])
set_word_select = set(df_word_select["word"])

In [14]:
not_in_sent_word_list = list(set_word_select.difference(set_result_word_count))
not_in_sent_word_list

['merhaba', 'devam', 'bence', 'gerçek', 'et', 'pekala', 'adam', 'asla']

In [15]:
df_youtube_sent_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment2/Result/\
2-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
1,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
3,çok güzel bir dakika bir dakika bir dakika bir...,525,531,çok güzel bir dakika bir dakika bir dakika bir...,OZhgDD1ScBg,https://www.youtube.com/watch?v=OZhgDD1ScBg&t=...
4,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
...,...,...,...,...,...,...
187595,o da o da,7102,7103,ha o da o da iptal peki peki,8DYpvzyEVg0,NaN
187596,ol o o bu,244,250,abone ol o o bu çocuğumuz sünnet ettirme ne iz...,oixYiG0tGgo,NaN
187597,o ne o o,3947,3948,o ne o o i lyasın yanındaki ayşen mi ya,D6Va3TXwFEQ,NaN
187598,ol o o o,2258,2260,abone ol o o o,K0b_9Xzy79A,NaN


In [33]:
df_not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_result, "search_string", 20000, simple=False)
#df_not_in_sent_selected_word_list_result = not_in_sent_selected_word_list_result.drop(["word"], axis=1)
df_not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,merhaba,merhaba merhaba merhaba merhaba merhaba uzun ve,517,522,dostlar merhaba merhaba merhaba merhaba merhab...,VRPrnvtZjvg,https://www.youtube.com/watch?v=VRPrnvtZjvg&t=...
1,merhaba,çok güzel burada sen burada merhaba sen bizim,1887,1891,uğraşıyor survivorda merhaba serman seni görme...,9GbYnL0tvcg,https://www.youtube.com/watch?v=9GbYnL0tvcg&t=...
2,merhaba,gel gel hadi gel hadi teşekkürler merhaba,6758,6760,gel gel hadi gel hadi teşekkürler merhaba jale,bt7g4VoVBoE,https://www.youtube.com/watch?v=bt7g4VoVBoE&t=...
3,merhaba,değilim bu peki onları burada merhaba onu,2151,2155,gidişat hakkında şu an emin değilim bu peki on...,1iByM08MvqU,https://www.youtube.com/watch?v=1iByM08MvqU&t=...
4,merhaba,var burada burada göre olabilir merhaba,99,102,hoparlörleri deyince sınıfından kaan var burad...,JrcKPsflI5k,https://www.youtube.com/watch?v=JrcKPsflI5k&t=99s
...,...,...,...,...,...,...,...
6733,asla,ama bu asla bu,194,196,ortadaki durumda en sekilde bilmiyorum bundan ...,3sSYXQnu6AU,NaN
6734,asla,ve bir de asla,9,11,ve bir de asla unutamayacaklarım,dBqFkaX130Q,NaN
6735,asla,ve onu asla bu,4790,4792,ve onu asla bu evliliğin olmayacağına ikna etm...,X1fpSLDLbFg,NaN
6736,asla,ve beni asla o,11568,11569,ve beni asla o sonsuzluğa sokamayacaksınız,kulZ_sWzBR0,NaN


In [32]:
#not_in_sent_selected_word_list_result.groupby("word")["video_id"].apply(list)

In [43]:
df_not_in_sent_selected_videoid_list = df_not_in_sent_selected_word_list_result.groupby("word")[["video_id"]].agg(lambda x: set(x))
df_not_in_sent_selected_videoid_list.reset_index(inplace=True)
df_not_in_sent_selected_videoid_list.sort_values(by="video_id", ascending=False, inplace=True, key=lambda x:x.str.len())
df_not_in_sent_selected_videoid_list.reset_index(drop=True, inplace=True)
df_not_in_sent_selected_videoid_list

,word,video_id
0,bence,"{b079OoXNVUA, 2DzH7cHQPsE, 0PObnwYVPLw, d73g1X..."
1,adam,"{fY6FOPbqQl0, n5eIuJEEdgs, HOFjdBDqknQ, kJHnaK..."
2,et,"{oRYKnD_df5M, frBM3E7xXlI, 3ZFfL3wl1so, vYQWj2..."
3,devam,"{frBM3E7xXlI, 3ZFfL3wl1so, QVoFWmBZyZI, vYQWj2..."
4,asla,"{a_SoPbglmYg, HOFjdBDqknQ, u2OguxxvCoY, EcfG7z..."
5,gerçek,"{gYoIugaLSJw, 8hbHNROHIvA, u2OguxxvCoY, hX8S64..."
6,merhaba,"{Rm6zwwnAVeA, jgAT53lbnVI, bsQD5xMYc7o, pvcitd..."
7,pekala,"{Wp7JJK2jToU, HhrBZuxjFxM, d9tZZGbLQMU, qmMZmy..."


In [48]:
videoid_list_group_list = df_not_in_sent_selected_videoid_list["video_id"].to_list()
videoid_list_group_list

[{'b079OoXNVUA',
  '2DzH7cHQPsE',
  '0PObnwYVPLw',
  'd73g1XrUYxE',
  'IbkoW6_fesk',
  '7G29Odeyqpk',
  'R5By7Dmyfwo',
  'zI2VldmdTM4',
  'g1Vag7OgCXw',
  '96DG6V_P9Yk',
  'FUVrye2RlnI',
  'hX8S64wZSDU',
  'sX7jAW0onSE',
  'f332DVN3EoQ',
  'PQP98H0RqDQ',
  'RRZzlj4qGCw',
  'IqAwin7WV6E',
  'VWHKv7pkLiE',
  'R02ljlzvh_A',
  'LwmbPWjegxQ',
  'UCzXULyScAU',
  'earwXiKop_w',
  '16gxQE8C1p0',
  'JfEl_wc7WiE',
  '9iBhLq7p_Xk',
  'BDUqLR_dd8Q',
  'EO4RHGzuFsw',
  'u1x5Br5cftc',
  'LmY6NZwmSa4',
  'ZybViRe2e6o',
  'MQ_nenSkjbo',
  'fzMD9GjvcSY',
  'bUr_cw89CRo',
  'hS1Jn4JeHzs',
  'XNBOk8QO8I0',
  '61v7xkFFvFA',
  'eJrrDQe1scs',
  'j5XOzTRnvjg',
  'oYykJu6A7Dg',
  'dXYd77Gevj8',
  'WcRHKUhrmHQ',
  'khMON7k5kWk',
  '30Sd_IpkgK4',
  'twGX5mN6Mcs',
  'i7RrNTLaaGE',
  'nVuWSoZDUfQ',
  'TsieXYhREDc',
  'rVhb6OMlips',
  'wEc24CZ97kI',
  'uXWqKxbROyY',
  'D_yb9_19sEo',
  '8Yu_uR8rZrA',
  'VBdcD7hLGuo',
  'eHtgG748qhI',
  '2SgmBwBSt8M',
  '5DpBItxxFws',
  'FvvNyw0607o',
  '_LOeU9XVjr4',
  'q_WSTbal5z8

In [56]:
set_list = []
for i in range(len(df_not_in_sent_selected_videoid_list)-2):
    var_1 = videoid_list_group_list[i]
    set_list.append(var_1)
set_intersect = set.intersection(*set_list)
list_intersect = list(set_intersect) 
list_intersect   

['kzxUQG7yJTU',
 'MhHCQ_cb3T8',
 '3LDpq53oV0o',
 'o1BNtH2rqY0',
 '6NFjjfvQ7ZQ',
 '8GCGRdwg80Y',
 'kulZ_sWzBR0']

In [57]:
df_intersect_result = df_not_in_sent_selected_word_list_result[df_not_in_sent_selected_word_list_result["video_id"].isin(list_intersect)]
df_intersect_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
24,merhaba,evet efendim tekrar merhaba,8132,8134,evet efendim tekrar merhaba,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...
193,devam,tamam tamam devam et devam et,2786,2788,tamam tamam devam et devam et,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...
315,devam,böyle devam et tamam mı,3273,3275,böyle devam et tamam mı,8GCGRdwg80Y,https://www.youtube.com/watch?v=8GCGRdwg80Y&t=...
365,devam,biraz daha böyle devam,4397,4399,biraz daha böyle devam edersen bir şeyin var s...,3LDpq53oV0o,https://www.youtube.com/watch?v=3LDpq53oV0o&t=...
509,devam,öyle de devam ederim,6106,6108,ben bu yaşıma kadar tek başıma geldim öyle de ...,6NFjjfvQ7ZQ,NaN
...,...,...,...,...,...,...,...
6459,asla,sana asla böyle bir,41,43,sana asla böyle bir konuda yalan söylemem,o1BNtH2rqY0,NaN
6532,asla,asla böyle bir şey,2683,2685,meral asla böyle bir şey talep etmedi etmez de,8GCGRdwg80Y,NaN
6577,asla,bir daha sizi asla,382,384,bir daha sizi asla odamın etrafında görmek ist...,3LDpq53oV0o,NaN
6731,asla,ya asla olmaz o,3029,3031,ya asla olmaz o geliyorsa ben de geliyorum,MhHCQ_cb3T8,NaN


In [58]:
df_intersect_result["video_url"] = "https://www.youtube.com/watch?v="+df_intersect_result['video_id'].map(str)+"&t="+df_intersect_result['start_time'].map(str)+"s"
df_intersect_result

<ipython-input-58-23f5f6eb5eaf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_intersect_result["video_url"] = "https://www.youtube.com/watch?v="+df_intersect_result['video_id'].map(str)+"&t="+df_intersect_result['start_time'].map(str)+"s"


,word,search_string,start_time,end_time,sentence,video_id,video_url
24,merhaba,evet efendim tekrar merhaba,8132,8134,evet efendim tekrar merhaba,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...
193,devam,tamam tamam devam et devam et,2786,2788,tamam tamam devam et devam et,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...
315,devam,böyle devam et tamam mı,3273,3275,böyle devam et tamam mı,8GCGRdwg80Y,https://www.youtube.com/watch?v=8GCGRdwg80Y&t=...
365,devam,biraz daha böyle devam,4397,4399,biraz daha böyle devam edersen bir şeyin var s...,3LDpq53oV0o,https://www.youtube.com/watch?v=3LDpq53oV0o&t=...
509,devam,öyle de devam ederim,6106,6108,ben bu yaşıma kadar tek başıma geldim öyle de ...,6NFjjfvQ7ZQ,https://www.youtube.com/watch?v=6NFjjfvQ7ZQ&t=...
...,...,...,...,...,...,...,...
6459,asla,sana asla böyle bir,41,43,sana asla böyle bir konuda yalan söylemem,o1BNtH2rqY0,https://www.youtube.com/watch?v=o1BNtH2rqY0&t=41s
6532,asla,asla böyle bir şey,2683,2685,meral asla böyle bir şey talep etmedi etmez de,8GCGRdwg80Y,https://www.youtube.com/watch?v=8GCGRdwg80Y&t=...
6577,asla,bir daha sizi asla,382,384,bir daha sizi asla odamın etrafında görmek ist...,3LDpq53oV0o,https://www.youtube.com/watch?v=3LDpq53oV0o&t=...
6731,asla,ya asla olmaz o,3029,3031,ya asla olmaz o geliyorsa ben de geliyorum,MhHCQ_cb3T8,https://www.youtube.com/watch?v=MhHCQ_cb3T8&t=...


In [67]:
# Option
#df_intersect_sent_count = df_intersect_result.groupby("search_string")[["search_string"]].count()
df_intersect_sent_count = df_intersect_result.groupby(["search_string","start_time","end_time","sentence","video_id","video_url"])[["search_string"]].count()
df_intersect_sent_count.rename(columns={"search_string":"count"}, inplace=True)
df_intersect_sent_count.reset_index(inplace=True)
df_intersect_sent_count.sort_values(by="count", ascending=False, inplace=True)
df_intersect_sent_count

,search_string,start_time,end_time,sentence,video_id,video_url,count
47,sen böyle devam et,4747,4749,sen böyle devam et,3LDpq53oV0o,https://www.youtube.com/watch?v=3LDpq53oV0o&t=...,2
29,değil mi devam et,3612,3613,değil mi devam et,8GCGRdwg80Y,https://www.youtube.com/watch?v=8GCGRdwg80Y&t=...,2
51,tamam tamam devam et devam et,2786,2788,tamam tamam devam et devam et,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...,2
24,böyle devam et tamam mı,3273,3275,böyle devam et tamam mı,8GCGRdwg80Y,https://www.youtube.com/watch?v=8GCGRdwg80Y&t=...,2
28,devam et tamam mı,2334,2336,eğer sana iyi gelen buysa devam et tamam mı,kzxUQG7yJTU,https://www.youtube.com/watch?v=kzxUQG7yJTU&t=...,2
...,...,...,...,...,...,...,...
26,daha ne kadar devam,822,824,daha ne kadar devam edecektin bu yalana,o1BNtH2rqY0,https://www.youtube.com/watch?v=o1BNtH2rqY0&t=...,1
27,daha ne kadar devam,1862,1863,bakalım yalan söylemeye daha ne kadar devam ed...,o1BNtH2rqY0,https://www.youtube.com/watch?v=o1BNtH2rqY0&t=...,1
30,değilim sana hiç değilim zaten bence,9756,9759,şakacı değilim sana hiç değilim zaten bence,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...,1
31,evet efendim tekrar merhaba,8132,8134,evet efendim tekrar merhaba,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...,1


In [76]:
df_intersect_sent_word_join = df_intersect_result.groupby(["search_string","start_time","end_time","sentence","video_id","video_url"])["word"].apply(lambda x: ','.join(x)).reset_index()
df_intersect_sent_word_join.drop_duplicates(inplace=True)
df_intersect_sent_word_join.sort_values(by="word", ascending=False, inplace=True, key=lambda x:x.str.len())
df_intersect_sent_word_join

,search_string,start_time,end_time,sentence,video_id,video_url,word
19,bu adam asla böyle bir,3604,3606,bu adam asla böyle bir ifade vermez,o1BNtH2rqY0,https://www.youtube.com/watch?v=o1BNtH2rqY0&t=...,"adam,asla"
47,sen böyle devam et,4747,4749,sen böyle devam et,3LDpq53oV0o,https://www.youtube.com/watch?v=3LDpq53oV0o&t=...,"devam,et"
24,böyle devam et tamam mı,3273,3275,böyle devam et tamam mı,8GCGRdwg80Y,https://www.youtube.com/watch?v=8GCGRdwg80Y&t=...,"devam,et"
51,tamam tamam devam et devam et,2786,2788,tamam tamam devam et devam et,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...,"devam,et"
29,değil mi devam et,3612,3613,değil mi devam et,8GCGRdwg80Y,https://www.youtube.com/watch?v=8GCGRdwg80Y&t=...,"devam,et"
...,...,...,...,...,...,...,...
58,yardım et sen de bize,7710,7712,bırak çantanı yardım et sen de bize,o1BNtH2rqY0,https://www.youtube.com/watch?v=o1BNtH2rqY0&t=...,et
12,benim et o zaman ben de,5177,5179,ah canım benim et o zaman ben de salatayı hazı...,MhHCQ_cb3T8,https://www.youtube.com/watch?v=MhHCQ_cb3T8&t=...,et
48,sen yardım et şu bizim,434,436,sen yardım et şu bizim oğlana,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...,et
46,sen bana yardım et,4382,4384,sanki hiç geçmeyecekmiş gibi allahım sen bana ...,6NFjjfvQ7ZQ,https://www.youtube.com/watch?v=6NFjjfvQ7ZQ&t=...,et


In [ ]:
df_intersect_sent_word_join("")

In [68]:
df_not_in_sent_selected_word_remain_result = word_in_wordgroup(["merhaba","pekala"], df_youtube_sent_result, "search_string", 10, simple=False)
#df_not_in_sent_selected_word_remain_result = not_in_sent_selected_word_list_result.drop(["word"], axis=1)
df_not_in_sent_selected_word_remain_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,merhaba,merhaba merhaba merhaba merhaba merhaba uzun ve,517,522,dostlar merhaba merhaba merhaba merhaba merhab...,VRPrnvtZjvg,https://www.youtube.com/watch?v=VRPrnvtZjvg&t=...
1,merhaba,çok güzel burada sen burada merhaba sen bizim,1887,1891,uğraşıyor survivorda merhaba serman seni görme...,9GbYnL0tvcg,https://www.youtube.com/watch?v=9GbYnL0tvcg&t=...
2,merhaba,gel gel hadi gel hadi teşekkürler merhaba,6758,6760,gel gel hadi gel hadi teşekkürler merhaba jale,bt7g4VoVBoE,https://www.youtube.com/watch?v=bt7g4VoVBoE&t=...
3,merhaba,değilim bu peki onları burada merhaba onu,2151,2155,gidişat hakkında şu an emin değilim bu peki on...,1iByM08MvqU,https://www.youtube.com/watch?v=1iByM08MvqU&t=...
4,merhaba,var burada burada göre olabilir merhaba,99,102,hoparlörleri deyince sınıfından kaan var burad...,JrcKPsflI5k,https://www.youtube.com/watch?v=JrcKPsflI5k&t=99s
5,merhaba,efendim ben de merhaba tabii ki efendim,4230,4233,bakabilir miyim efendim ben de merhaba tabii k...,8OjNg0Sspro,https://www.youtube.com/watch?v=8OjNg0Sspro&t=...
6,merhaba,merhaba çok teşekkür ederim bu benim,2249,2253,hepsini değiştirir sen yemezsin ali merhaba ço...,nPVd4XaeM34,https://www.youtube.com/watch?v=nPVd4XaeM34&t=...
7,merhaba,merhaba çok teşekkür ederim buraya,3275,3278,merhaba tolga merhaba çok teşekkür ederim bura...,AvfpnsqaWw4,https://www.youtube.com/watch?v=AvfpnsqaWw4&t=...
8,merhaba,sadece merhaba merhaba o kadar ama,5574,5577,arkadaşım değil büyük hanım aynı bölükteydik s...,VBdcD7hLGuo,https://www.youtube.com/watch?v=VBdcD7hLGuo&t=...
9,merhaba,biz sadece bir merhaba demek için,5612,5614,biz sadece bir merhaba demek için geldik,MWMmHcPKvfs,https://www.youtube.com/watch?v=MWMmHcPKvfs&t=...


In [ ]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])
df_file_count.sort_values(by="word_count", ascending=False)
df_file_count

In [ ]:
df_check = df_file_count[df_file_count["word_count"] > 2]
df_check

In [ ]:
check_list = df_check["word"].to_list()
check_list

In [ ]:
len(check_list)

In [ ]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > 2]
check_list = df_check["word"].to_list()
remove_list = []
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file = df_file[df_file["search_string"] != f"{var_sent}"]
        df_file_count = word_count_result(df_file, df_file.columns[0:1])
        df_check = df_file_count[df_file_count["word_count"] > 2]
        check_list = df_check["word"].to_list()
        remove_list.append(var_sent)
    else:
        df_file = df_file

In [ ]:
df_file

In [ ]:
remove_list

In [ ]:
word_count_result(df_file, df_file.columns[0:1])

In [ ]:
test = ["aet","önce"]

In [ ]:
test in check_list

In [ ]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])  # "search_string" column
df_check = df_file_count[df_file_count["word_count"] > 2]
check_list = df_check["word"].to_list()
i = 0
while i < len(df_file):
    var_sent in df_file.loc[i,"search_string"]
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    if all(result_list):
        df_file = df_file[df_file["search_string"] != f"{var_sent}"]
        df_file_count = word_count_result(df_file, df_file.columns[0:1])
        df_check = df_file_count[df_file_count["word_count"] > 1]
        check_list = df_check["word"].to_list()
    else:
        df_file = df_file
    i += 1

In [ ]:
df_file

In [ ]:
check_list

In [ ]:
len(check_list)

In [ ]:
while var_sent in df_file["search_string"]:
    print(var_sent)

In [ ]:
all_list = []
for var_sent in df_file["search_string"]:
    result_list = []
    words = re.findall(r"\w+",var_sent, re.UNICODE)
    for word in words:
        if word in check_list:
            result_list.append(True)
        else:
            result_list.append(False)
    all_list.append(result_list)

In [ ]:
all_list

In [ ]:
test_list = []
for j in all_list:
    if all(j):
        test_list.append(j)
    else:
        pass

In [ ]:
test_list

In [ ]:
result_list

In [ ]:
all(result_list)

In [ ]:
a = []

In [ ]:
a.append(True)

In [ ]:
a.append(False)

In [ ]:
a